In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# 定义数据预处理
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载CIFAR-10数据集
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 定义模型
#参数pretrained表示是否加载预训练的权重。如果pretrained为True，那么模型将加载在ImageNet数据集上预训练的权重。
# 这些预训练的权重可以提供更好的初始权重，有助于模型在其他任务上进行迁移学习。如果pretrained为False，
# 则使用随机初始化的权重进行训练。
model = models.resnet18(pretrained=False)
num_classes = 10
model.fc = nn.Linear(512, num_classes)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 训练模型
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # 前向传播和计算损失
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")
# 在测试集上评估模型
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy}%')
torch.cuda.empty_cache()

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10, Loss: 1.7536
Epoch 2/10, Loss: 1.4616
Epoch 3/10, Loss: 1.3217
Epoch 4/10, Loss: 1.2295
Epoch 5/10, Loss: 1.1596
Epoch 6/10, Loss: 1.0971
Epoch 7/10, Loss: 1.0381
Epoch 8/10, Loss: 0.9983
Epoch 9/10, Loss: 0.9562
Epoch 10/10, Loss: 0.9238
Epoch [10/10], Test Accuracy: 67.21%


In [3]:
import torch
import torch.nn as nn
#模型的保存和加载
model=nn.Linear(3,1)
#修改权重和偏置后保存模型
new_weight = torch.tensor([[1.0, 2.0, 3.0]])
new_bias = torch.tensor([4.0])
model.weight = nn.Parameter(new_weight)
model.bias = nn.Parameter(new_bias)
torch.save(model.state_dict(),"./model.pt")

newModel=nn.Linear(3,1)
print("默认参数",newModel.weight,newModel.bias)
newModel.load_state_dict(torch.load("./model.pt"))
print("加载后",newModel.weight,newModel.bias)


默认参数 Parameter containing:
tensor([[-0.4357, -0.0781,  0.0136]], requires_grad=True) Parameter containing:
tensor([-0.2013], requires_grad=True)
加载后 Parameter containing:
tensor([[1., 2., 3.]], requires_grad=True) Parameter containing:
tensor([4.], requires_grad=True)
